In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value
import pandas as pd
import numpy as np


In [ ]:
# Sample data
service_areas = ['Area1', 'Area2', 'Area3']
demand = [20, 30, 15]  # Demand in each service area
driver_capacities = [50, 40]  # Capacities of two drivers
travel_distances = {
    ('Driver1', 'Area1'): 10,
    ('Driver1', 'Area2'): 15,
    ('Driver1', 'Area3'): 12,
    ('Driver2', 'Area1'): 8,
    ('Driver2', 'Area2'): 20,
    ('Driver2', 'Area3'): 18
}


In [ ]:
# Initialize the model
model = LpProblem(name="driver_allocation", sense=LpMaximize)


In [ ]:
# Define decision variables
driver_to_area = LpVariable.dicts("DriverToArea",
                                  [(f"Driver{i+1}", area) for i in range(len(driver_capacities)) for area in service_areas],
                                  cat='Binary')


In [ ]:
# Define objective function
model += lpSum(driver_to_area[(f"Driver{i+1}", area)] * demand[idx] for idx, area in enumerate(service_areas)
               for i in range(len(driver_capacities)))


In [ ]:
# Constraints
# Each service area must be covered by exactly one driver
for area in service_areas:
    model += lpSum(driver_to_area[(f"Driver{i+1}", area)] for i in range(len(driver_capacities))) == 1

# Driver capacity constraints
for i, capacity in enumerate(driver_capacities):
    model += lpSum(driver_to_area[(f"Driver{i+1}", area)] * demand[idx] for idx, area in enumerate(service_areas)) <= capacity


In [ ]:
# Solve the model
model.solve()

# Print optimal allocation
for v in model.variables():
    if v.varValue > 0:
        print(f"{v.name}: {v.varValue}")
